# Day 13
# Part 1

In [84]:
from collections import namedtuple
import math


Timetable = namedtuple('Timetable', 'timestamp bus_ids')


def parse_data(s):
    lines = s.strip().splitlines()
    return Timetable(int(lines[0]), [int(bus_id) for bus_id in lines[1].split(',') if bus_id != 'x'])

In [85]:
test_data = parse_data('''939
7,13,x,x,59,x,31,19
''')

test_data

Timetable(timestamp=939, bus_ids=[7, 13, 59, 31, 19])

Divide the timestamp by the bus id, round it up, multiply by the bus id to get the first timestamp you can get that bus. Then get the minimum of those and calculate the answer.

In [86]:
def part_1(data):
    earliest_time, earliest_bus = min(
        (math.ceil(data.timestamp / bus_id) * bus_id - data.timestamp, bus_id)
        for bus_id in data.bus_ids
    )
    return earliest_time * earliest_bus

In [87]:
assert part_1(test_data) == 295

In [88]:
data = parse_data(open('input').read())
part_1(data)

2305

## Part 2

In [89]:
def parse_data_2(s):
    lines = s.strip().splitlines()
    return {
        (i, int(bus_id))
        for i, bus_id in enumerate(lines[1].split(','))
        if bus_id != 'x'
    }

In [90]:
test_data_2 = parse_data_2('''939
7,13,x,x,59,x,31,19
''')

test_data_2

{(0, 7), (1, 13), (4, 59), (6, 31), (7, 19)}

I can't think of a mathematically clever way of doing this, so loop using the largest bus id and hope that it doesn't take too long.

In [91]:
import itertools


def part_2(data):
    t_delta, bus_id = max(data, key=lambda x: x[1])
    # Make it marginally faster as we don't have to check the
    # bus_id above
    checks = data - {(t_delta, bus_id)}
    timestamps = (bus_id * i - t_delta for i in itertools.count(1))
    
    for t in timestamps:
        if all(
            (t + t_delta) % bus == 0
            for t_delta, bus in checks
        ):
            return t
        
        
assert part_2(test_data_2) == 1068781

In [92]:
data_2 = parse_data_2(open('input').read())

In [124]:
# Don't run this
# part_2(data_2)

Hopefully that will finish by tomorrow.

Perhaps not, though I'll leave it running in pypy, just in case (edit: it finished after twelve hours).
```
(advent) [mike@marge day13]$ time pypy3 day13_part2.py 
552612234243498

real	730m52.797s
user	730m14.714s
sys	0m2.002s
```

Just looking at two buses at a time, are the coinciding timestamps regular?

In [93]:
def valid_timestamps(bus_id_a, t_delta_a, bus_id_b, t_delta_b):
    timestamps = (bus_id_a * i - t_delta_a for i in itertools.count(1))
    for t in timestamps:
        if (t + t_delta_b) % bus_id_b == 0:
            yield t
            
for t in itertools.islice(valid_timestamps(7, 0, 13, 1), 10):
    print(t)

77
168
259
350
441
532
623
714
805
896


Yes, as they're primes, as all the ids seem to be. These two buses can be combined into one with an id of 91 (the multiple of the two ids as they're prime) and a time delta of $91-77=14$. Let's try doing that for all the buses.

In [116]:
import functools

# use (time delta, id) tuple to represent the buses
def combine_buses(bus_a, bus_b):
    time_delta_a, bus_id_a = bus_a
    time_delta_b, bus_id_b = bus_b    
    vt = valid_timestamps(bus_id_a, time_delta_a, bus_id_b, time_delta_b)
    bus_id = bus_id_a * bus_id_b
    time_delta = bus_id - next(vt)
    return (time_delta, bus_id)

def part_2_opt(data):
    time_delta, big_bus_id = functools.reduce(combine_buses, data)
    return big_bus_id - time_delta

def part_2_debug(data):
    data = list(data)
    next_bus = combine_buses(data[0], data[1])
    print(f'{data[0]} and {data[1]} combine to make {next_bus}')
    for bus in data[2:]:
        print(f'{next_bus} and {bus} combine to make {(next_bus := combine_buses(next_bus, bus))}')
    return next_bus

In [117]:
part_2_opt(test_data_2)

1068781

In [96]:
6970 + 4199

11169

In [110]:
easy_data = parse_data_2('''-1
2,3,x,x,5''')

part_2_debug(easy_data)

(4, 5) and (0, 2) combine to make (4, 10)
(4, 10) and (1, 3) combine to make (4, 30)


(4, 30)

In [118]:
part_2(easy_data)

26

In [119]:
part_2_opt(easy_data)

26

In [121]:
part_2_opt(data_2)

552612234243498

In [122]:
%%timeit
part_2_opt(data_2)

79.6 µs ± 939 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
